In [291]:
import pandas as pd

CHECK DATA

In [292]:
df = pd.read_csv('../dataset/train.csv')
df.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,15.748,19.292,11.739,20.155,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,17.717,17.893,12.748,17.889,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,16.738,18.240,12.718,18.288,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,18.576,18.339,12.583,19.060,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,16.990,15.746,11.306,18.093,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


PREPROCESSING

In [293]:
df = df.drop('id', axis=1)
df = df.drop('product_code', axis=1)

In [294]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform='pandas')

ohetransform = ohe.fit_transform(df[['attribute_0', 'attribute_1']])
ohetransform

df = pd.concat([df, ohetransform], axis=1).drop(columns=['attribute_0', 'attribute_1'])

In [295]:
df['attribute_1_material_7'] = 0.0

In [296]:
df.isnull().sum()

loading                    250
attribute_2                  0
attribute_3                  0
measurement_0                0
measurement_1                0
measurement_2                0
measurement_3              381
measurement_4              538
measurement_5              676
measurement_6              796
measurement_7              937
measurement_8             1048
measurement_9             1227
measurement_10            1300
measurement_11            1468
measurement_12            1601
measurement_13            1774
measurement_14            1874
measurement_15            2009
measurement_16            2110
measurement_17            2284
failure                      0
attribute_0_material_5       0
attribute_0_material_7       0
attribute_1_material_5       0
attribute_1_material_6       0
attribute_1_material_8       0
attribute_1_material_7       0
dtype: int64

In [297]:
from sklearn.impute import KNNImputer

features = ['loading', 'measurement_3', 'measurement_4', 
            'measurement_5', 'measurement_6', 'measurement_7', 
            'measurement_8', 'measurement_9', 'measurement_10', 
            'measurement_11', 'measurement_12', 'measurement_13', 
            'measurement_14', 'measurement_15', 'measurement_16', 
            'measurement_17']

imputer = KNNImputer(n_neighbors=5)
df[features] = imputer.fit_transform(df[features])

CREATE MODEL

In [298]:
X = df.drop('failure', axis=1)
y = df['failure']

In [299]:
from imblearn.under_sampling import NearMiss

undersample = NearMiss(n_neighbors=1)

X, y = undersample.fit_resample(X, y)

In [300]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)
X_scaled_df.head()

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,attribute_0_material_5,attribute_0_material_7,attribute_1_material_5,attribute_1_material_6,attribute_1_material_8,attribute_1_material_7
0,0.009928,-1.181438,0.526219,0.992850,-0.271223,-1.368665,-0.840740,0.146197,0.762069,0.351882,0.366955,-2.644914,0.114943,0.580223,-0.567730,-0.994685,1.792209,0.355047,1.792044,1.184425,-1.638541,-0.485489,0.485489,-0.798071,-0.502958,1.202050,0.0
1,0.404136,-0.500527,-0.846336,-0.608504,0.506381,0.600623,0.213912,0.427478,-0.034180,-0.511196,-0.514700,0.708665,-0.351323,-1.152253,-0.619991,-0.439502,-0.218079,-0.858985,0.640999,-0.732488,-0.264228,-0.485489,0.485489,1.253022,-0.502958,-0.831912,0.0
2,-1.037888,0.861293,0.526219,-0.875397,-0.789626,0.600623,0.000272,0.718100,0.203204,0.093345,-0.055395,-0.245204,0.243061,-1.573563,0.735923,0.190530,0.625942,-0.152547,0.536816,-0.290276,0.348453,2.059779,-2.059779,1.253022,-0.502958,-0.831912,0.0
3,0.224574,1.542203,-1.532613,1.526635,-1.567230,-0.712235,0.304579,0.880018,-0.436563,0.047254,0.147333,-0.911193,-0.660066,-1.117207,-0.810420,-0.116856,-0.202154,-0.388991,-0.673865,-0.359108,-0.736756,-0.485489,0.485489,-0.798071,-0.502958,1.202050,0.0
4,0.397230,-0.500527,-0.846336,0.459065,1.283985,0.600623,1.068473,-1.067149,-1.208328,-1.035345,1.136686,-0.464336,-1.097977,1.480745,0.401025,-0.322506,-1.262567,0.769365,-0.490647,0.285522,-1.076537,-0.485489,0.485489,1.253022,-0.502958,-0.831912,0.0


In [301]:
from sklearn.ensemble import RandomForestRegressor

random_forest = RandomForestRegressor(n_estimators=1000, max_depth=7, random_state=42)

random_forest.fit(X_scaled_df, y)

RandomForestRegressor(max_depth=7, n_estimators=1000, random_state=42)

PREPARE RESULT

In [302]:
df_test = pd.read_csv('../dataset/test.csv')
df_test.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,measurement_7,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,19.305,10.178,17.534,18.168,11.598,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,17.883,11.927,17.228,16.033,11.179,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,18.475,10.481,16.619,18.189,12.126,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,16.518,10.888,15.293,18.592,11.304,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,17.808,12.693,17.678,15.814,13.431,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044


In [303]:
id = df_test['id']
df_test = df_test.drop('id', axis=1)
df_test = df_test.drop('product_code', axis=1)

In [304]:
df_test.isnull().sum()

loading            223
attribute_0          0
attribute_1          0
attribute_2          0
attribute_3          0
measurement_0        0
measurement_1        0
measurement_2        0
measurement_3      329
measurement_4      409
measurement_5      508
measurement_6      624
measurement_7      720
measurement_8      846
measurement_9      904
measurement_10    1067
measurement_11    1136
measurement_12    1240
measurement_13    1303
measurement_14    1440
measurement_15    1542
measurement_16    1678
measurement_17    1740
dtype: int64

In [305]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output = False).set_output(transform='pandas')

ohetransform = ohe.fit_transform(df_test[['attribute_0', 'attribute_1']])
ohetransform

df_test = pd.concat([df_test, ohetransform], axis=1).drop(columns=['attribute_0', 'attribute_1'])

In [306]:
df_test['attribute_1_material_8'] = 0.0

In [307]:
from sklearn.impute import KNNImputer

features = ['loading', 'measurement_3', 'measurement_4', 
            'measurement_5', 'measurement_6', 'measurement_7', 
            'measurement_8', 'measurement_9', 'measurement_10', 
            'measurement_11', 'measurement_12', 'measurement_13', 
            'measurement_14', 'measurement_15', 'measurement_16', 
            'measurement_17']

imputer = KNNImputer(n_neighbors=5)
df_test[features] = imputer.fit_transform(df_test[features])

In [308]:
import pandas as pd

column_order = X_scaled_df.columns

df_test = df_test.reindex(columns=column_order)

In [309]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_test_scaled = scaler.fit_transform(df_test)
df_test_scaled_2 = pd.DataFrame(df_test_scaled, index=df_test.index, columns=df_test.columns)

In [310]:
pd.set_option('display.max_columns', None)

In [311]:
test_result = random_forest.predict(df_test_scaled_2)

SAVING RESULT

In [312]:
df_result = pd.DataFrame({
    'id': id,
    'failure': test_result
})

In [314]:
df_result.to_csv('../result/rfr.csv', index=False)